In [14]:
# %pip install torch torchvision torchaudio

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50

In [16]:
INPUT_DIR_A = '../data/model_input_a'

In [17]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the batch size
batch_size = 8

# Define the transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.RandomRotation(10),  # Apply random rotation with a maximum angle of 10 degrees
    transforms.ToTensor()  # Convert the image to a tensor
])

# Define the dataset
dataset = datasets.ImageFolder(INPUT_DIR_A, transform=transform)

# Verify the classes
classes = dataset.classes
print(classes)

dataset = DataLoader(dataset, batch_size=batch_size, shuffle=True)


['Monet', 'Picasso']


In [18]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Device: {device}")

# Load the pre-trained ResNet model
model = resnet50(pretrained=True)

# Freeze all the parameters in the pre-trained model
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes in your dataset
num_classes = len(classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Define the number of training epochs
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Set the model to training mode
    model.train()
    
    for images, labels in dataset:
        # Move the images and labels to the GPU if available
        images = images.to(device)
        labels = labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        # Update the running loss
        running_loss += loss.item()
        
        # Compute the predicted labels
        _, predicted = torch.max(outputs.data, 1)
        
        # Update the total and correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    # Compute the accuracy
    accuracy = 100 * correct / total
    
    # Print the loss and accuracy for each epoch
    print(f"Epoch {epoch+1}/{num_epochs}: Loss = {running_loss:.4f}, Accuracy = {accuracy:.2f}%")


Device: cpu


Epoch 1/10: Loss = 2.0350, Accuracy = 63.16%
Epoch 2/10: Loss = 2.0174, Accuracy = 57.89%
Epoch 3/10: Loss = 1.5385, Accuracy = 78.95%
Epoch 4/10: Loss = 1.4117, Accuracy = 84.21%
Epoch 5/10: Loss = 1.5728, Accuracy = 89.47%
Epoch 6/10: Loss = 0.9194, Accuracy = 94.74%
Epoch 7/10: Loss = 0.8901, Accuracy = 94.74%
Epoch 8/10: Loss = 0.7643, Accuracy = 94.74%
Epoch 9/10: Loss = 0.7267, Accuracy = 94.74%
Epoch 10/10: Loss = 0.5990, Accuracy = 100.00%
